In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import csv
import time
from datetime import datetime

In [2]:
path = "D://Documents//NGU//Year 4//S1//CV//Project_test"

# Collect Data

In [3]:
cap = cv2.VideoCapture(0)

trained_data = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

Id = input("Your ID: ")

count = 0

while True:
    
    _, frame = cap.read()
    
    gray_scale2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = trained_data.detectMultiScale(gray_scale2, 1.3, 5)
    
    for x,y,w,h in faces:
        
        count = count + 1
        
        cv2.imwrite("D://Documents//NGU//Year 4//S1//CV//Project_test//Dataset" + "//User." + str(Id)+"."+str(count)+".jpg", gray_scale2[y:y+h, x:x+w])
        
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
        
    cv2.imshow("Camera", frame)
    
    k = cv2.waitKey(1)
    
    if count > 600:
        break
    
cap.release()
cv2.destroyAllWindows()
print("Completed!")

Your ID: 100200
Completed!


# Model Training

In [8]:
# Local Binary Patterns Histograms (LBPH) is a popular face recognition algorithm that uses local binary patterns to extract features from images to recognize faces 
trainer = cv2.face.LBPHFaceRecognizer_create()

In [9]:
def GetImgID(path):
    imagesPath = [os.path.join(path, i) for i in os.listdir(path)]
    
    Faces = []
    IDs = []
    
    for img in imagesPath:
        face = Image.open(img).convert("L")
        faceNumpy = np.array(face, "uint8")
        
        #Extract Id
        Id = os.path.split(img)[-1].split(".")[1]
        Id = int(Id)
        Faces.append(faceNumpy)
        IDs.append(Id)
        
        cv2.imshow("Training Progress....", faceNumpy)
        cv2.waitKey(1)
        
    return IDs, Faces

In [5]:
Ids, faces = GetImgID(path+"//Dataset")
trainer.train(faces, np.array(Ids))
trainer.write("Trainer.yml")
cv2.destroyAllWindows()

# Testing & Record attendance and duration in an excel sheet

In [46]:
# The columns names in the excel sheet
Column_Names = ["ID", "Duration", "Status", "Arrival Time"]


# Dictionary that hold all the captured students:
# Every student got captured will be stored in this dictionary in the following format:
# {Id: [0, 1, 2, 3, 4]})
# 0 -> Entered_time
# 1 -> Exit_time (initially it will be "", and will be updated when he leaves the class)
# 2 -> count from which we will determine wheather he is in or out the class, if even, then he was in and will leave the class
# 3 -> Duration spent in the lecture, this will be accumelated during the session, as when he leaves the timer stops once he enter again the timer continues.
# 4 -> Arrival time for the student, will use it just in the excel sheet
Students = {}

# Access the Camera
cap = cv2.VideoCapture(0)

# We used the pre-trained classifier used for face detection in OpenCV
trained_data = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


trainer = cv2.face.LBPHFaceRecognizer_create()

# Import Trained Model, which we get from the training part
trainer.read("Trainer.yml")

# Dictionary to store the last attendance time for each person, so that we gave him a time to sit instead of taking him multiple times
last_attendance_time = {}


while True:
    
    try:
        # initiate the camera
        _, frame = cap.read()
        
        gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = trained_data.detectMultiScale(gray_scale, 1.3, 5)

        for x, y, w, h in faces:
            
            # Input the grayscale image (face) to the model
            Id, conf = trainer.predict(gray_scale[y:y+h, x:x+w])

            if conf > 50:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 1)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 50, 255), 2)
                cv2.rectangle(frame, (x, y-40), (x+w, y), (50, 50, 255), -1)
                cv2.putText(frame, str(Id), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

                # Get the time
                tnow = time.time()
                date = datetime.fromtimestamp(tnow).strftime("%d-%m-%Y")
                timest = datetime.fromtimestamp(tnow).strftime("%H:%M:%S")

                # Update the last attendance time for this person and add him to the students dictionary
                if Id not in last_attendance_time:
                    last_attendance_time[Id] = tnow
                
                if Id not in Students:
                    Students[Id] = [timest, "", 0, 0, timest]
                
                #Check if attendance was already taken for this person in the last 10 seconds
                if Id in last_attendance_time and tnow - last_attendance_time[Id] < 10:
                    continue

                else:
                        """
                        So, I checked if the count value is even. If it is, 
                        this means that the person was in the class and will now leave. 
                        Then, I got the current time, which will be the exit time for the person. 
                        After that, I calculated the duration the person spent in the class 
                        by subtracting the enter time from the exit time. 
                        I added the duration to the total duration the person spent in the class and then increment 
                        the count value. Finally, I reset the enter and exit time until the person gets in again.
                        
                        If the count value is odd, it means that I was out of the class and have now returned. 
                        So, I will get the current time, which will be my entered time. 
                        Then, I will increment the count value by one to indicate that I have entered the class again.
                        """
                        # Student is going out
                        lis = Students[Id]
                        if lis[2] % 2 == 0:
                            tnow = time.time()
                            exit = datetime.fromtimestamp(tnow).strftime("%H:%M:%S")
                            entry = datetime.strptime(lis[0], "%H:%M:%S")
                            exit = datetime.strptime(exit, "%H:%M:%S")
                            duration = exit - entry
                            lis[1] = exit.strftime("%H:%M:%S")
                            lis[3] = lis[3] + (duration.seconds / 60)
                            lis[2] = lis[2] + 1
                            lis[0] = ""
                            lis[1] = ""
                            Students[Id] = lis
                            last_attendance_time[Id] = tnow
                        else:
                            # Student is back in
                            lis = Students[Id]
                            tnow = time.time()
                            enter = datetime.fromtimestamp(tnow).strftime("%H:%M:%S")
                            lis[0] = enter
                            lis[2] = lis[2] + 1
                            Students[Id] = lis
                            last_attendance_time[Id] = tnow

            else:
                # If the student is not registered, his attendance will not be recorded
                cv2.putText(frame, "Not Registered", (x, y-40), cv2.FONT_ITALIC, 1, (50, 50, 255), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 50, 255), 1)

        cv2.imshow("Camera", frame)
        
    except Exception as e:
        print(e)
        break

    try:
        # If we hit q key, this means that the lecture has ended and i will push the Student data stored in the dictionary into an excel shhet
        if cv2.waitKey(1) == ord('q'):
            # Open the CSV file for writing
            with open("D://Documents//NGU//Year 4//S1//CV//Project_test//Attendance//Attendance_"+str(date)+".csv", 'w', newline='') as file:
                writer = csv.writer(file)

                # Write the header row
                writer.writerow(Column_Names)

                # Write the data rows
                for id, data in Students.items():
                    duration = data[3]
                    status = "Attended" if (duration) > 2 else "Not Fully Attended"
                    writer.writerow([id, duration, status, data[4]])
                file.close()
            break
    except Exception as e:
        print(e)
        break

cap.release()
cv2.destroyAllWindows()
print("Completed!")


0.0
0.28511500358581543
0.5722486972808838
0.5802483558654785
0.6672122478485107
0.7242882251739502
0.8052637577056885
0.8651070594787598
0.9424772262573242
1.018183946609497
1.0291850566864014
1.0826570987701416
1.1641955375671387
1.238304853439331
1.3173127174377441
1.3978643417358398
1.5360026359558105
1.6108098030090332
1.6642255783081055
1.6747581958770752
1.7211105823516846
1.731109619140625
1.787574291229248
1.8601274490356445
1.9235916137695312
1.9823846817016602
2.037738800048828
2.113269567489624
2.178325891494751
2.2633073329925537
2.3363447189331055
2.4665966033935547
2.477597236633301
2.539048910140991
2.550055980682373
2.602409601211548
2.6635098457336426
2.713590383529663
2.7739710807800293
2.823098659515381
2.868752956390381
2.912548065185547
2.922966718673706
2.969143867492676
3.008530855178833
3.020946741104126
3.083339214324951
3.095339059829712
3.1695258617401123
3.1805319786071777
3.245593547821045
3.426748037338257
3.4914891719818115
3.556108236312866
3.5656316280

In [25]:
Students

{100200: ['15:20:58', '', 0, 0]}

In [35]:
tnow = time.time()
exit = datetime.fromtimestamp(tnow).strftime("%H:%M:%S")
entry = datetime.strptime("15:20:58", "%H:%M:%S")
exit = datetime.strptime(exit, "%H:%M:%S")
duration = exit - entry
print(type(duration.seconds))

<class 'int'>


In [34]:
if not Students[100200]:
    print("No")
else:
    print("Yes")

Yes


In [36]:
12//60

0